In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append("../libs")
import pandas as pd
from data_manager.etf_data_manager import etf_data_iter, save_res_df_to_windows, get_etf_data_by_symbols

from config import DataPath
import datetime
import numpy as np

/home/gouzi/projects/invest/.venv/lib/python3.12/site-packages/py_mini_racer/py_mini_racer.py:13: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
from factors.ma import SlopeRiskRatio
from factors.new_high import NewHigh
from factors.portfolio.correlation import CorrelationFactor
from factors.average_true_range import AverageTrueRange

In [3]:
from fetcher.etf import get_all_etf_code
all_codes = get_all_etf_code()


HTTPSConnectionPool(host='88.push2.eastmoney.com', port=443): Read timed out.


In [4]:
name_dict = all_codes[["代码", "名称"]].set_index("代码").to_dict().get("名称", {})

In [6]:
hold_list = [
    "159206",
    "560860",
    "159201",
    "516570",
    "159326",
    "513750",
    "159697",
    "560280",
    "513800",
    "159516"
]
all_money = 309104

In [7]:
def risk_concentration(corr_matrix):
    """风险集中度分析"""
    # 平均相关系数（衡量整体相关性）
    avg_corr = corr_matrix.values[np.triu_indices(len(corr_matrix), 1)].mean()
    
    # 相关性矩阵的特征值分析（衡量多元化程度）
    eigenvalues = np.linalg.eigvalsh(corr_matrix)
    eigenvalue_ratio = eigenvalues.max() / eigenvalues.sum()
    
    # 有效组合数（衡量真正独立的资产数量）
    if eigenvalues.sum() > 0:
        effective_n = (eigenvalues.sum()**2) / (eigenvalues**2).sum()
    else:
        effective_n = 0
    
    return {
        'average_correlation': avg_corr,
        'dominant_eigenvalue_ratio': eigenvalue_ratio,
        'effective_diversification_number': effective_n,
        'portfolio_concentration_score': eigenvalue_ratio * avg_corr
    }

In [8]:
def get_k_ratio_res(k):
    all_res = []
    for df in etf_data_iter():
        df.add_factors(SlopeRiskRatio())
        df.name = name_dict.get(df.symbol, "")
        res = df.calc_factors().tail(1)["SlopeRiskRatio"].values[0]
        all_res.append([df.symbol, df.name,res])
    save_res_df_to_windows(pd.DataFrame(all_res, columns=["code", "name", "ratio"]).sort_values(by="ratio", ascending=False), 
                           f"斜率风险比/{datetime.date.today().strftime('%Y%m%d')}-{k}倍.xlsx")

In [18]:
def generate_new_high_list(hold_list):
    add_to_list = []
    sell_list = []
    today_dir = os.path.join(DataPath.DEFAULT_WINDOWS_PATH, "新高", datetime.datetime.today().strftime("%Y%m%d"))
    if not os.path.exists(today_dir):
        os.mkdir(today_dir)
    for data in etf_data_iter():
        data.add_factors(NewHigh())
        data.calc_factors()
        data.name = name_dict[data.symbol]
        if 1 in data.factor_results["NewHigh"].iloc[-1:].values:
            add_to_list.append((data.symbol, data.name))
            data.output_with_factors_to(today_dir)
        elif data.symbol in hold_list:
            data.output_with_factors_to(today_dir)
            if data.factor_results["NewHigh"].iloc[-1] == -1:
                sell_list.append((data.symbol, data.name))
    if sell_list:
        pd.DataFrame(sell_list, columns=["symbol", "name"]).to_csv(os.path.join(today_dir, "sell.csv"), index=False, encoding="utf-8-sig")
    pd.DataFrame(add_to_list, columns=["symbol", "name"]).to_csv(os.path.join(today_dir, "add.csv"), index=False, encoding="utf-8-sig")
    return add_to_list

In [10]:
def analysis_corr_between_add_and_hold(add_to, hold):
    hold_datas = get_etf_data_by_symbols(hold)
    add_datas = get_etf_data_by_symbols([i[0] for i in add_to])
    idx = []
    corr_analysis = []
    for num, add in enumerate(add_datas):
        idx.append(add.symbol)
        corr = CorrelationFactor()
        corrs_res = risk_concentration(corr(*([hold_data for hold_data in hold_datas] + [add])))
        corrs_res["name"] = add_to[num][1]
        corr_analysis.append(corrs_res)
    final_analysis_res = pd.DataFrame(corr_analysis, index=idx).sort_values(by="effective_diversification_number", ascending=False)
    today_dir = os.path.join(DataPath.DEFAULT_WINDOWS_PATH, "新高", datetime.datetime.today().strftime("%Y%m%d"))
    final_analysis_res.to_excel(os.path.join(today_dir, "corr_analysis.xlsx"), index=True)
    return final_analysis_res

In [11]:
get_k_ratio_res(1)

  0%|          | 0/1326 [00:00<?, ?it/s]

In [19]:
add_to_list = generate_new_high_list(hold_list)

  0%|          | 0/1326 [00:00<?, ?it/s]

In [21]:
analysis_corr_between_add_and_hold(add_to_list, hold_list)

,average_correlation,dominant_eigenvalue_ratio,effective_diversification_number,portfolio_concentration_score,name
511820,0.439847,0.546094,3.045271,0.240198,鹏华添利ETF
511160,0.432445,0.546271,3.042772,0.236232,国债ETF东财
511970,0.429753,0.546337,3.041939,0.234790,国寿货币ETF
511880,0.420382,0.547707,3.028600,0.230246,银华日利ETF
518600,0.461906,0.549284,3.012562,0.253717,上海金ETF
...,...,...,...,...,...
159677,0.560466,0.620366,2.454776,0.347694,1000增强ETF
516120,0.559619,0.620869,2.451380,0.347450,化工50ETF
516020,0.559774,0.620957,2.451161,0.347596,化工ETF
159944,0.560211,0.621564,2.444555,0.348207,材料ETF


In [22]:
for etf in get_etf_data_by_symbols(hold_list + [i[0] for i in add_to_list]):
    etf.add_factors(AverageTrueRange())
    res = etf.calc_factors()
    curr_atr = res.iloc[-1,-1]
    max_position = all_money * 0.003 / curr_atr
    print(f"Can buy etf {etf.symbol} {name_dict.get(etf.symbol)} no more than {max_position}")

Can buy etf 159206 卫星ETF no more than 19814.358974358965
Can buy etf 560860 工业有色ETF no more than 22862.721893491107
Can buy etf 159201 自由现金流ETF no more than 64040.88397790049
Can buy etf 516570 化工行业ETF no more than 51289.38053097344
Can buy etf 159326 电网设备ETF no more than 28834.328358208935
Can buy etf 513750 港股通非银ETF no more than 27931.084337349403
Can buy etf 159697 油气ETF no more than 44411.494252873694
Can buy etf 560280 工程机械ETF no more than 35339.63414634144
Can buy etf 513800 日本东证指数ETF no more than 49325.10638297868
Can buy etf 159516 半导体设备ETF no more than 21932.63954588459
Can buy etf 159870 化工ETF no more than 58839.593908629395
Can buy etf 159516 半导体设备ETF no more than 21932.63954588459
Can buy etf 516650 有色金属ETF基金 no more than 20264.685314685314
Can buy etf 515880 通信ETF no more than 8241.308211873447
Can buy etf 588240 科创200ETF指数 no more than 29382.50950570343
Can buy etf 518680 金ETF no more than 9761.178947368435
Can buy etf 159381 创业板人工智能ETF华夏 no more than 14663.377609108158
C

In [41]:
all_money * 0.003

918.0219900000001